In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/drive/MyDrive/COMP9517_ZXCZH /content/

Mounted at /content/drive


# EfficientNet

In [ ]:
!pip install efficientnet_pytorch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from efficientnet_pytorch import EfficientNet
from tqdm import tqdm

# device layout
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

# dataset allocation
train_dir = '/content/COMP9517_ZXCZH/train'
test_dir = '/content/COMP9517_ZXCZH/test'

# Data enhancement pipeline
train_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# data loading
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, num_workers=4, pin_memory=True)

# Model initialization
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=15)
model.to(device)

# Training configuration
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

# Initialize GradScaler before the training loop
scaler = torch.cuda.amp.GradScaler()

for epoch in range(10):
    model.train()
    train_loss, correct, total = 0, 0, 0
    with tqdm(train_loader, unit="batch") as pbar:
        pbar.set_description(f"Epoch {epoch+1}/10")
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            # Enable automatic mixing precision context
            with torch.cuda.amp.autocast():
                outputs = model(images)       # FP16/FP32 is automatically selected
                loss = criterion(outputs, labels)

            # Scale the gradient and backpropagate
            scaler.scale(loss).backward()

            # Update parameters (automatic unscale gradient)
            scaler.step(optimizer)

            # Update the pantograph status
            scaler.update()

            # statistical information
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            pbar.set_postfix({
                "loss": f"{train_loss/(total//64):.3f}",
                "acc": f"{100*correct/total:.1f}%"
            })

    # verification stage
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    # Print the epoch summary
    print(f"Val Acc: {100*val_correct/val_total:.1f}%")
    print("-" * 50)

# Overfitting judgment
final_train_acc = 100 * correct / total
final_val_acc = 100 * val_correct / val_total
print(f"\nTraining accuracy: {final_train_acc:.1f}% | Verification accuracy: {final_val_acc:.1f}%")
if final_train_acc - final_val_acc > 10:
    print("Overfitting detected!")
else:
    print("Normal training")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 235MB/s]
<ipython-input-4-9dce2438cec1>:53: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


Loaded pretrained weights for efficientnet-b0


Epoch 1/10:   0%|          | 0/150 [00:00<?, ?batch/s]<ipython-input-4-9dce2438cec1>:66: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1/10: 100%|██████████| 150/150 [14:11<00:00,  5.67s/batch, loss=0.672, acc=83.9%]


Val Acc: 95.2%
--------------------------------------------------


...

Val Acc: 97.9%
--------------------------------------------------


Epoch 9/10: 100%|██████████| 150/150 [01:27<00:00,  1.71batch/s, loss=0.037, acc=98.8%]


Val Acc: 97.6%
--------------------------------------------------


Epoch 10/10: 100%|██████████| 150/150 [01:27<00:00,  1.72batch/s, loss=0.031, acc=99.0%]


Val Acc: 97.4%
--------------------------------------------------

Training accuracy: 99.0% | Verification accuracy: 97.4%
Normal training


In [ ]:
!pip install grad-cam

# Show grad_cam results

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import random

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
# The last convolution layer of EfficientNet is usually model._blocks[-1]._project_conv
target_layers = [model._blocks[-1]._project_conv]
# Obtain the category name and the original image path
class_names = train_dataset.classes
image_paths, labels = zip(*test_dataset.samples)

for class_idx, class_name in enumerate(class_names):
    print(f"Grad-CAM for {class_name}...")

    # Obtain all image paths of this category
    class_image_paths = [path for path, label in zip(image_paths, labels) if label == class_idx]
    if not class_image_paths:
        print(f"No images found for class {class_name}")
        continue

    # Randomly select a picture
    img_path = random.choice(class_image_paths)
    pil_img = Image.open(img_path).convert("RGB")
    input_tensor = test_transforms(pil_img).unsqueeze(0).to(device)

    # Obtain the model prediction category
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
        pred_class = output.argmax(dim=1).item()

    # Grad-CAM Instantiate and generate a heat map
    cam = GradCAM(model=model, target_layers=target_layers)
    grayscale_cam = cam(input_tensor=input_tensor, targets=[ClassifierOutputTarget(pred_class)])[0]

    # Convert the input image to numpy format to overlay the heat map
    img_np = np.array(pil_img.resize((224, 224))) / 255.0
    visualization = show_cam_on_image(img_np, grayscale_cam, use_rgb=True)

    # results
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.title(f"Original - Label: {class_name}")
    plt.imshow(img_np)
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title(f"Grad-CAM - Predicted: {class_names[pred_class]}")
    plt.imshow(visualization)
    plt.axis('off')
    plt.show()


Output hidden; open in https://colab.research.google.com to view.